In [ ]:
# Genre & Style Comment Generator — Reusable Template

import pandas as pd
from openai import OpenAI
import os
import time
from tenacity import retry, wait_fixed, stop_after_attempt, retry_if_exception_type, before_sleep_log
from dotenv import load_dotenv
import logging

# --- LOGGING SETUP FOR RETRIES --- #
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# --- CONFIGURATION --- #
INPUT_FILE = "../data/playlist.csv"            # Change this for new playlists
OUTPUT_FILE = "../output/playlist_output.csv"

load_dotenv("../.env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found. Add it to the .env file in the project root.")

# --- SETUP --- #
client = OpenAI(api_key=OPENAI_API_KEY)

# --- LOAD DATA (resume from output if it exists) --- #
if os.path.exists(OUTPUT_FILE):
    print(f"📂 Resuming from {OUTPUT_FILE}")
    df = pd.read_csv(OUTPUT_FILE)
else:
    df = pd.read_csv(INPUT_FILE)

# Ensure the comment column exists
if "comment" not in df.columns:
    df["comment"] = ""

# --- FUNCTION TO GENERATE COMMENT WITH RETRIES --- #
@retry(
    wait=wait_fixed(5),
    stop=stop_after_attempt(3),
    retry=retry_if_exception_type(Exception),
    before_sleep=before_sleep_log(logger, logging.INFO)
)
def generate_genre_comment(title, artist):
    prompt = f"""
    Describe the track for DJs in one line. Include decade and country if possible. Use vivid club-friendly phrasing.
    Avoid repeating genre terms and keep the tone colorful and stylish.
    Format: <Main genre>; <production traits>; <scene/mood/era info>.

    Track title: "{title}"
    Artist: {artist}
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a music genre expert and DJ selector."},
            {"role": "user", "content": prompt.strip()}
        ]
    )
    return response.choices[0].message.content.strip()

# --- PROCESS EACH TRACK --- #
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)
total = len(df)
skipped = 0

for i, row in df.iterrows():
    if pd.notna(row["comment"]) and str(row["comment"]).strip():
        skipped += 1
        continue
    print(f"🎧 Track {i+1}/{total}: {row['title']} by {row['artist']}")
    df.at[i, "comment"] = generate_genre_comment(row["title"], row["artist"])
    # Save after each track so progress is not lost
    df.to_csv(OUTPUT_FILE, index=False)
    time.sleep(1.5)

if skipped:
    print(f"⏭️  Skipped {skipped} tracks that already had comments")
print(f"✅ Done! Comments saved to {OUTPUT_FILE}")